In [44]:
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import time
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.svm import LinearSVC
import pickle

In [2]:
sent= pd.read_csv("./Outputs/sentimentos_tweeteros.csv")
sent.head()

,date,user_name,user_loc,message,full_name,country,country_code,geo_code,longitud_mensaje,palabras,refinado,latitud,longitud,states,Polaridad,Subjetividad,Label
0,2020-10-10 11:41:36,susanha77835097,"Florida, USA",@blossomingabyss @JoeBiden He filled his pocke...,"Sunny Isles Beach, FL",United States,US,[-80.125071 25.92906 ],140,18,"['filled', 'pocketskids', 'familyharris', 'unq...",-80.125071,25.929060,FL,0.40,0.90,Positiva
1,2020-10-10 11:19:05,tomborelli,"Greenwich, CT",Supporting @JoeBiden means packing the Supreme...,"Greenwich, CT",United States,US,[-73.63941 41.050217],144,17,"['supporting', 'means', 'packing', 'supreme', ...",-73.639410,41.050217,CT,0.25,0.25,Positiva
2,2020-10-10 09:39:10,John_Di_Lemme,(561) 847-3467,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"West Palm Beach, FL",United States,US,[-80.12262 26.721896],133,15,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",-80.122620,26.721896,FL,0.00,0.00,Neutral
3,2020-10-10 08:39:14,Parnell_100,United Kingdom,@JoeBiden There is NO STOPPING this AMERICAN T...,"Belfast, Northern Ireland",United Kingdom,GB,[-5.928413 54.595869],140,23,"['stopping', 'american', 'trinity', 'train', '...",-5.928413,54.595869,Non_USA,0.00,0.00,Neutral
4,2020-10-10 07:18:33,sqlblues,"Weston-super-Mare, England",@FenrirWolf26 @Stanhope2011VJ Britons have als...,"Weston-super-Mare, England",United Kingdom,GB,[-2.94513 51.346796],139,17,"['britons', 'also', 'pumped', 'relentless', 'd...",-2.945130,51.346796,Non_USA,-0.30,0.40,Negativa


In [3]:
le = LabelEncoder()
sent["Label_enc"] = le.fit_transform(sent["Label"])

# Display the encoded labels
display(sent[["Label_enc"]].head())

# Select the features and the target
X = sent['refinado']
y = sent["Label_enc"]

,Label_enc
0,2
1,2
2,1
3,1
4,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
tfidfstops=stopwords.words('english')
vectorizer = TfidfVectorizer(max_features=5000, stop_words= tfidfstops)

# Create the tf-idf vectorizer
model_vectorizer = TfidfVectorizer()

# First fit the vectorizer with our training set
tfidf_train = vectorizer.fit_transform(X_train)

# Now we can fit our test data with the same vectorizer
tfidf_test = vectorizer.transform(X_test)

# Initialize the Bernoulli Naive Bayes classifier
nb = BernoulliNB()

# Fit the model
nb.fit(tfidf_train, y_train)

# Print the accuracy score
best_accuracy = cross_val_score(nb, tfidf_test, y_test, cv=10, scoring='accuracy').max()
print("Accuracy:",best_accuracy)



Accuracy: 0.52


### Precision del 52%, un poco mejor que lanzar una moneda al aire y elegir un resultado. 
# HAY QUE MEJORARLO.

In [6]:
y_pred = nb.predict(tfidf_test)

# Print the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix\nMatriz 3x3 donde se ven los tweets que se catalogaron erroneamente\n")
print(cm)

# Print the Classification Report
cr = classification_report(y_test, y_pred)
print("\n\nClassification Report\n")
print(cr)

Confusion Matrix
Matriz 3x3 donde se ven los tweets que se catalogaron erroneamente

[[ 29  66  17]
 [  0 225   7]
 [  1  59  97]]


Classification Report

              precision    recall  f1-score   support

           0       0.97      0.26      0.41       112
           1       0.64      0.97      0.77       232
           2       0.80      0.62      0.70       157

    accuracy                           0.70       501
   macro avg       0.80      0.62      0.63       501
weighted avg       0.77      0.70      0.67       501



In [7]:
pickle.dump(nb, open("./Outputs/model.pkl", 'wb'))

In [8]:
train=pd.read_csv("./Outputs/sentimentos_tweeteros.csv")
train.head()
test=pd.read_csv("./Outputs/sentimentos_tweeteros.csv")
train.head()


,date,user_name,user_loc,message,full_name,country,country_code,geo_code,longitud_mensaje,palabras,refinado,latitud,longitud,states,Polaridad,Subjetividad,Label
0,2020-10-10 11:41:36,susanha77835097,"Florida, USA",@blossomingabyss @JoeBiden He filled his pocke...,"Sunny Isles Beach, FL",United States,US,[-80.125071 25.92906 ],140,18,"['filled', 'pocketskids', 'familyharris', 'unq...",-80.125071,25.929060,FL,0.40,0.90,Positiva
1,2020-10-10 11:19:05,tomborelli,"Greenwich, CT",Supporting @JoeBiden means packing the Supreme...,"Greenwich, CT",United States,US,[-73.63941 41.050217],144,17,"['supporting', 'means', 'packing', 'supreme', ...",-73.639410,41.050217,CT,0.25,0.25,Positiva
2,2020-10-10 09:39:10,John_Di_Lemme,(561) 847-3467,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"West Palm Beach, FL",United States,US,[-80.12262 26.721896],133,15,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",-80.122620,26.721896,FL,0.00,0.00,Neutral
3,2020-10-10 08:39:14,Parnell_100,United Kingdom,@JoeBiden There is NO STOPPING this AMERICAN T...,"Belfast, Northern Ireland",United Kingdom,GB,[-5.928413 54.595869],140,23,"['stopping', 'american', 'trinity', 'train', '...",-5.928413,54.595869,Non_USA,0.00,0.00,Neutral
4,2020-10-10 07:18:33,sqlblues,"Weston-super-Mare, England",@FenrirWolf26 @Stanhope2011VJ Britons have als...,"Weston-super-Mare, England",United Kingdom,GB,[-2.94513 51.346796],139,17,"['britons', 'also', 'pumped', 'relentless', 'd...",-2.945130,51.346796,Non_USA,-0.30,0.40,Negativa


In [9]:
def numerar_polaridad(polaridad):
    if polaridad == 'Positiva':
        return 1
    if polaridad == "Neutral":
        return 0
    if polaridad == "Negativa":
        return -1

train["nLabel"] = train["Label"].apply(numerar_polaridad)
test["nLabel"] = test["Label"].apply(numerar_polaridad)

In [10]:
test.drop(['date','Label','user_loc','Polaridad','Subjetividad','full_name','longitud_mensaje','palabras','country_code','geo_code','country','states','latitud','longitud'],axis=1 ,inplace=True)
train.drop(['date','Label','user_loc','Polaridad','Subjetividad','full_name','longitud_mensaje','palabras','country_code','geo_code','country','states','latitud','longitud'],axis=1 ,inplace=True)


In [11]:
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(train['message'][:2000], train['nLabel'][:2000], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            stop_words='english',
                            lowercase=True)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

# this is where we define the values for GridSearchCV to iterate over
parameters = {'bow__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'classifier__alpha': (1e-2, 1e-3),
             }

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Best Model: 0.680625 using {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}


Mean: 0.669375 Stdev:(0.026583) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.680625 Stdev:(0.030033) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.663750 Stdev:(0.019922) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.674375 Stdev:(0.023791) with: {'bow__ngram_range': (1, 1), 'classifier__alpha': 0.001, 'tfidf__use_idf': False}
Mean: 0.643750 Stdev:(0.030491) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.658750 Stdev:(0.038951) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.646250 Stdev:(0.030644) with: {'bow__ngram_range': (1, 2), 'classifier__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.655625 Stdev:(0.034737) with: {'bow__ngram_range'

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    5.6s finished


In [12]:
import joblib
joblib.dump(grid, "./models/twitter_sentiment.pkl")

['./models/twitter_sentiment.pkl']

In [13]:
# load from file and predict using the best configs found in the CV step
model_NB = joblib.load("./models/twitter_sentiment.pkl")

# get predictions from best model above
y_preds = model_NB.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(classification_report(y_test, y_preds))



accuracy score:  0.6775


confusion matrix: 
 [[ 43  23  25]
 [  6 142  28]
 [ 10  37  86]]


              precision    recall  f1-score   support

          -1       0.73      0.47      0.57        91
           0       0.70      0.81      0.75       176
           1       0.62      0.65      0.63       133

    accuracy                           0.68       400
   macro avg       0.68      0.64      0.65       400
weighted avg       0.68      0.68      0.67       400



In [14]:
tweet_preds = model_NB.predict(sent['message'])

# append predictions to dataframe
df_tweet_preds = sent.copy()
df_tweet_preds['predictions'] = tweet_preds
df_tweet_preds.shape

(2504, 19)

In [15]:
import random
# print text and sentiment

index = random.sample(range(tweet_preds.shape[0]), 20)
for text, sentiment in zip(df_tweet_preds.message[index],
                           df_tweet_preds.predictions[index]):
    print (sentiment, '--', text, '\n')

0 -- @rolandsmartin @Jillbiden46 @JoeBiden Biden was expos d to Trump and should be in quarantine. This is a grave error. 

0 -- @Jim_Jordan @realDonaldTrump @SpeakerPelosi @JoeBiden @EliseStefanik @DevinNunes @SenSchumer… https://t.co/qYCGkHjogS 

1 -- @glennkirschner2 @Michell45129550 @JoeBiden Joe Biden, like President Obama, will have to spend a large amount of t… https://t.co/8BDF1hKyS6 

0 -- @gnuseibeh @realDonaldTrump @JoeBiden This is some bullshit. No one in the WH has followed the guidelines set and t… https://t.co/rvgMHCv5w4 

1 -- @DeAnna4Congress I would say you are. Biden at least shows concern for Donald Trump and Masks do work. 

0 -- Michelle Obama's Closing Argument | Joe Biden For President 2020 https://t.co/A7SBruuHoS via @MichelleObama @JoeBiden 

0 -- @MikeBloomberg @voteblackpac @UniteCountryPAC @JoeBiden @KamalaHarris Buden/Harris is where our vote needs to be. S… https://t.co/uqSfTrk0Zo 

-1 -- @ABC13News @realDonaldTrump @JoeBiden Trump is lying as usual!  Bi

In [16]:
train.head()

,user_name,message,refinado,nLabel
0,susanha77835097,@blossomingabyss @JoeBiden He filled his pocke...,"['filled', 'pocketskids', 'familyharris', 'unq...",1
1,tomborelli,Supporting @JoeBiden means packing the Supreme...,"['supporting', 'means', 'packing', 'supreme', ...",1
2,John_Di_Lemme,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",0
3,Parnell_100,@JoeBiden There is NO STOPPING this AMERICAN T...,"['stopping', 'american', 'trinity', 'train', '...",0
4,sqlblues,@FenrirWolf26 @Stanhope2011VJ Britons have als...,"['britons', 'also', 'pumped', 'relentless', 'd...",-1


In [17]:
#train=pd.get_dummies(data=train, columns=['nLabel'])

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def polarity_scores_all(tweet):
  '''
  Takes string of text to:
  1. Gets sentiment metrics
  2. Returns negative, neutral, positive 
  and compound scores as lists.
  '''
  neg, neu, pos, compound = [], [], [], []
  analyser = SentimentIntensityAnalyzer()
  
  for message in tweet:
    dict_ = analyser.polarity_scores(message)
    neg.append(dict_['neg'])
    neu.append(dict_['neu'])
    pos.append(dict_['pos'])
    compound.append(dict_['compound'])
  
  return neg, neu, pos, compound

In [19]:
all_scores = polarity_scores_all(train.refinado.values)
train['neg_scores'] = all_scores[0]
train['neu_scores'] = all_scores[1]
train['pos_scores'] = all_scores[2]
train['compound_scores'] = all_scores[3]

train.head(10)

,user_name,message,refinado,nLabel,neg_scores,neu_scores,pos_scores,compound_scores
0,susanha77835097,@blossomingabyss @JoeBiden He filled his pocke...,"['filled', 'pocketskids', 'familyharris', 'unq...",1,0.000,1.000,0.000,0.0000
1,tomborelli,Supporting @JoeBiden means packing the Supreme...,"['supporting', 'means', 'packing', 'supreme', ...",1,0.000,0.552,0.448,0.7579
2,John_Di_Lemme,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",0,0.247,0.753,0.000,-0.3182
3,Parnell_100,@JoeBiden There is NO STOPPING this AMERICAN T...,"['stopping', 'american', 'trinity', 'train', '...",0,0.167,0.833,0.000,-0.1531
4,sqlblues,@FenrirWolf26 @Stanhope2011VJ Britons have als...,"['britons', 'also', 'pumped', 'relentless', 'd...",-1,0.339,0.550,0.110,-0.5423
5,_walkforACure,US presidential debate officially cancelled\n\...,"['presidential', 'debate', 'officially', 'canc...",0,0.286,0.714,0.000,-0.2500
6,newollien,"@blossomingabyss @JoeBiden Oh behave, you tit....","['behave', 'tit', 'trump', 'tactic', 'debate',...",0,0.381,0.619,0.000,-0.5719
7,AIPCMaha,Joe Biden is surging in the battle for the Whi...,"['joe', 'biden', 'surging', 'battle', 'white',...",-1,0.206,0.794,0.000,-0.3818
8,RonnieMro,We know exactly what we’re getting with Donald...,"['know', 'exactly', 'getting', 'donald', 'trum...",-1,0.000,1.000,0.000,0.0000
9,cyn507,Looks like @realDonaldTrump is running scared....,"['looks', 'like', 'running', 'scared', 'bag', ...",0,0.341,0.465,0.194,-0.2263


In [20]:
pos_family = {  
    'NOUN' : ['NN','NNS','NNP'], # Removed 'NNPS'
    'PRON' : ['PRP','PRP$','WP'],
    'VERB' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'ADJ' :  ['JJ','JJR','JJS'],
    'ADV' : ['RB','RBR','RBS','WRB']
}

def count_pos_tag(tweets):
  '''
  Takes string of text to:
  1. Processes text and attaches POS tags
  2. Input the dictionary of POS tags into a Counter.
  2. Returns list of POS tags with occurrence number '''
  total_count = []
  for s in tweets:
    partial_count = {}
    s = s.split()
    count_pos = Counter(dict(nltk.pos_tag(s)).values())
    
    for item, value in count_pos.items():
      partial_count[item] = partial_count.get(item, 0) + 1
            
    total_count.append(partial_count)

  return total_count

In [21]:
# Retrieve POS tags with occurrence 
total_count = count_pos_tag(train.message.values)
# As dataframe 
pos_df = pd.DataFrame(total_count)
# Remove unwanted characters
# Inspection
pos_df.reindex

<bound method DataFrame.reindex of        RB  VBZ  PRP  VBD  PRP$  NNS   NN  VBN  PDT   DT  ...  NNPS  UH  RBR  \
0     1.0  1.0  1.0  1.0   1.0  1.0  1.0  1.0  1.0  1.0  ...   NaN NaN  NaN   
1     1.0  NaN  NaN  NaN   NaN  1.0  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   
2     NaN  NaN  NaN  NaN   NaN  1.0  1.0  NaN  NaN  NaN  ...   NaN NaN  NaN   
3     NaN  1.0  NaN  NaN   NaN  1.0  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   
4     1.0  NaN  NaN  NaN   NaN  1.0  1.0  1.0  NaN  1.0  ...   NaN NaN  NaN   
...   ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...   ...  ..  ...   
2499  1.0  NaN  1.0  1.0   NaN  NaN  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   
2500  NaN  1.0  1.0  NaN   NaN  NaN  1.0  NaN  1.0  1.0  ...   NaN NaN  NaN   
2501  NaN  NaN  1.0  1.0   NaN  NaN  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   
2502  NaN  NaN  NaN  NaN   NaN  NaN  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   
2503  1.0  NaN  1.0  1.0   NaN  1.0  1.0  NaN  NaN  1.0  ...   NaN NaN  NaN   

       (   $  LS

In [22]:

pos_df['NOUN'] = pos_df[pos_family['NOUN']].sum(axis=1)
pos_df['PRON'] = pos_df[pos_family['PRON']].sum(axis=1)
pos_df['VERB'] = pos_df[pos_family['VERB']].sum(axis=1)
pos_df['ADJ'] = pos_df[pos_family['ADJ']].sum(axis=1)
pos_df['ADV'] = pos_df[pos_family['ADV']].sum(axis=1)

pos_df = pos_df[['NOUN', 'PRON', 'VERB', 'ADJ', 'ADV']]

In [23]:
train = pd.concat([train, pos_df], axis = 1)
train = train.fillna(value=0.0)
train

,user_name,message,refinado,nLabel,neg_scores,neu_scores,pos_scores,compound_scores,NOUN,PRON,VERB,ADJ,ADV
0,susanha77835097,@blossomingabyss @JoeBiden He filled his pocke...,"['filled', 'pocketskids', 'familyharris', 'unq...",1,0.000,1.000,0.000,0.0000,3.0,2.0,4.0,0.0,1.0
1,tomborelli,Supporting @JoeBiden means packing the Supreme...,"['supporting', 'means', 'packing', 'supreme', ...",1,0.000,0.552,0.448,0.7579,3.0,0.0,2.0,1.0,1.0
2,John_Di_Lemme,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",0,0.247,0.753,0.000,-0.3182,3.0,0.0,1.0,1.0,0.0
3,Parnell_100,@JoeBiden There is NO STOPPING this AMERICAN T...,"['stopping', 'american', 'trinity', 'train', '...",0,0.167,0.833,0.000,-0.1531,3.0,0.0,2.0,0.0,0.0
4,sqlblues,@FenrirWolf26 @Stanhope2011VJ Britons have als...,"['britons', 'also', 'pumped', 'relentless', 'd...",-1,0.339,0.550,0.110,-0.5423,3.0,0.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,JojoAkoto,Great debate between Donald Trump and Joe Bide...,"['great', 'debate', 'donald', 'trump', 'joe', ...",1,0.000,0.305,0.695,0.9287,2.0,1.0,2.0,2.0,1.0
2500,Crackerthe4th,To all the undecided voters: the biggest sprea...,"['undecided', 'voters', 'big', 'spreader', 'mi...",0,0.339,0.403,0.258,0.0000,2.0,1.0,1.0,2.0,0.0
2501,ThomasCagganel1,@thebradfordfile Chris Wallace went after Dona...,"['chris', 'wallace', 'went', 'donald', 'trump'...",0,0.147,0.662,0.191,0.1531,2.0,1.0,2.0,1.0,0.0
2502,Hal4,https://t.co/faYZgsey1P Something #good for a ...,"['something', 'good', 'change']",1,0.000,0.408,0.592,0.4404,1.0,0.0,1.0,1.0,0.0


In [24]:
train.drop_duplicates(subset=['message'], inplace=True)

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2495 entries, 0 to 2503
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_name        2495 non-null   object 
 1   message          2495 non-null   object 
 2   refinado         2495 non-null   object 
 3   nLabel           2495 non-null   int64  
 4   neg_scores       2495 non-null   float64
 5   neu_scores       2495 non-null   float64
 6   pos_scores       2495 non-null   float64
 7   compound_scores  2495 non-null   float64
 8   NOUN             2495 non-null   float64
 9   PRON             2495 non-null   float64
 10  VERB             2495 non-null   float64
 11  ADJ              2495 non-null   float64
 12  ADV              2495 non-null   float64
dtypes: float64(9), int64(1), object(3)
memory usage: 272.9+ KB


In [26]:
train.head(10)

,user_name,message,refinado,nLabel,neg_scores,neu_scores,pos_scores,compound_scores,NOUN,PRON,VERB,ADJ,ADV
0,susanha77835097,@blossomingabyss @JoeBiden He filled his pocke...,"['filled', 'pocketskids', 'familyharris', 'unq...",1,0.000,1.000,0.000,0.0000,3.0,2.0,4.0,0.0,1.0
1,tomborelli,Supporting @JoeBiden means packing the Supreme...,"['supporting', 'means', 'packing', 'supreme', ...",1,0.000,0.552,0.448,0.7579,3.0,0.0,2.0,1.0,1.0
2,John_Di_Lemme,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",0,0.247,0.753,0.000,-0.3182,3.0,0.0,1.0,1.0,0.0
3,Parnell_100,@JoeBiden There is NO STOPPING this AMERICAN T...,"['stopping', 'american', 'trinity', 'train', '...",0,0.167,0.833,0.000,-0.1531,3.0,0.0,2.0,0.0,0.0
4,sqlblues,@FenrirWolf26 @Stanhope2011VJ Britons have als...,"['britons', 'also', 'pumped', 'relentless', 'd...",-1,0.339,0.550,0.110,-0.5423,3.0,0.0,2.0,1.0,1.0
5,_walkforACure,US presidential debate officially cancelled\n\...,"['presidential', 'debate', 'officially', 'canc...",0,0.286,0.714,0.000,-0.2500,2.0,1.0,1.0,1.0,1.0
6,newollien,"@blossomingabyss @JoeBiden Oh behave, you tit....","['behave', 'tit', 'trump', 'tactic', 'debate',...",0,0.381,0.619,0.000,-0.5719,2.0,1.0,4.0,1.0,1.0
7,AIPCMaha,Joe Biden is surging in the battle for the Whi...,"['joe', 'biden', 'surging', 'battle', 'white',...",-1,0.206,0.794,0.000,-0.3818,3.0,0.0,3.0,1.0,0.0
8,RonnieMro,We know exactly what we’re getting with Donald...,"['know', 'exactly', 'getting', 'donald', 'trum...",-1,0.000,1.000,0.000,0.0000,2.0,2.0,4.0,2.0,1.0
9,cyn507,Looks like @realDonaldTrump is running scared....,"['looks', 'like', 'running', 'scared', 'bag', ...",0,0.341,0.465,0.194,-0.2263,2.0,2.0,4.0,0.0,0.0


In [27]:
train.to_csv('./Outputs/feat_train_data.csv', index=False)

In [28]:
train.isnull().values.any()
train_prep=train.copy()
train_prep.head()

,user_name,message,refinado,nLabel,neg_scores,neu_scores,pos_scores,compound_scores,NOUN,PRON,VERB,ADJ,ADV
0,susanha77835097,@blossomingabyss @JoeBiden He filled his pocke...,"['filled', 'pocketskids', 'familyharris', 'unq...",1,0.000,1.000,0.000,0.0000,3.0,2.0,4.0,0.0,1.0
1,tomborelli,Supporting @JoeBiden means packing the Supreme...,"['supporting', 'means', 'packing', 'supreme', ...",1,0.000,0.552,0.448,0.7579,3.0,0.0,2.0,1.0,1.0
2,John_Di_Lemme,@ABC13News @realDonaldTrump @JoeBiden Nancy Pe...,"['nancy', 'pelosi', 'nuts', 'joe', 'biden', 'k...",0,0.247,0.753,0.000,-0.3182,3.0,0.0,1.0,1.0,0.0
3,Parnell_100,@JoeBiden There is NO STOPPING this AMERICAN T...,"['stopping', 'american', 'trinity', 'train', '...",0,0.167,0.833,0.000,-0.1531,3.0,0.0,2.0,0.0,0.0
4,sqlblues,@FenrirWolf26 @Stanhope2011VJ Britons have als...,"['britons', 'also', 'pumped', 'relentless', 'd...",-1,0.339,0.550,0.110,-0.5423,3.0,0.0,2.0,1.0,1.0


In [29]:
# Extracting Features
features = ['message', 'neu_scores', 'neg_scores', 'compound_scores', 'pos_scores']
label = ['nLabel']
# Saving features and label data in X and y for train-test split
X = train_prep[[col for col in train_prep.columns if col in features]]
y = train_prep[label]

X.shape, y.shape

((2495, 5), (2495, 1))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print("Shapes are {} and {}".format(X_train.shape, X_test.shape))

Shapes are (1746, 5) and (749, 5)


In [31]:
pd.Series(y_train.values.ravel()).value_counts(normalize=True)
pd.Series(y_train.values.ravel()).value_counts(normalize=True)

 0    0.457045
 1    0.310424
-1    0.232532
dtype: float64

## El dataset tiene un 45% de acertar si un mensaje es neutro, 31% si es positivo y un 23% de acertar si es negativo

In [32]:
# Helper functions

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [52]:
# Pipeline to convert tweets to a matrix of TF-IDF features.
tfidf = Pipeline([
                ('selector', TextSelector(key='message')),
                ('tfidf', TfidfVectorizer())
            ])

# Pipeline to convert tweets to a matrix of token counts
countvect = Pipeline([
                ('selector', TextSelector(key='message')),
                ('countvect', CountVectorizer())
            ])

In [53]:


# Applying tfidf anf countvec to features

neu_scores =  Pipeline([
                ('selector', NumberSelector(key='neu_scores')),
                ('minmax', MinMaxScaler())
            ])
neg_scores =  Pipeline([
                ('selector', NumberSelector(key='neg_scores')),
                ('minmax', MinMaxScaler())
            ])
pos_scores =  Pipeline([
                ('selector', NumberSelector(key='pos_scores')),
                ('minmax', MinMaxScaler())
            ])

compound_scores =  Pipeline([
                ('selector', NumberSelector(key='compound_scores')),
                ('minmax', MinMaxScaler())
            ])

In [54]:


# defining different sets of text processors
def features_union(textProcessor):
    return FeatureUnion([('text', textProcessor),
                      ('neu_scores', neu_scores),
                      ('neg_scores', neg_scores),
                      ('pos_scores', pos_scores),
                      ('compound_scores', compound_scores)])


In [55]:
le = LabelEncoder().fit(y_train.values.ravel())

y_train = le.transform(y_train.values.ravel())
y_test = le.transform(y_test.values.ravel())

X_train.shape, X_test.shape


AttributeError: 'numpy.ndarray' object has no attribute 'values'

## Clasificador Bayes

In [56]:
# instantiate classifier
clf = MultinomialNB()

# combine features
features_count = features_union(countvect)

# define pipeline object 
nb_pipeline = Pipeline([('features', features_count),
                       ('nb', clf)])

# Fit classifier
nb_pipeline.fit(X_train, y_train)

# score
nb_pipeline.score(X_test, y_test)

0.6995994659546061

### Support vector machine

In [57]:
# instantiate classifier
svm = LinearSVC()

#  combine features
features_tfidf = features_union(tfidf)

# define pipeline object
svm_pipeline = Pipeline([('features', features_tfidf),
                       ('svm', svm)])

# Fit classifier
svm_pipeline.fit(X_train, y_train)

# score
svm_pipeline.score(X_test, y_test)

0.7543391188251002

## Cross-validation

In [ ]:


# Naive Bayes Classifier

# combine features
features_tfidf = features_union(tfidf)

# instantiate pipeline object
nb_pipeline = Pipeline([('feats', features_tfidf),  ('clf', MultinomialNB())])

# parameter grid (3x3x2x2x3x3x2) combinations
parameters = {
    'feats__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'feats__text__tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)), 
    'feats__text__tfidf__use_idf': (False, True),
    'feats__text__tfidf__binary':(False, True),
    'feats__text__tfidf__binary':('l1', 'l2', None),
    'clf__alpha': (1.0, 5.0, 10.0),
    'clf__fit_prior': (True, False),     
}

# instantiate GridSearchCV object with pipeline and parameters with 3-folds cross-validation
nb_grid = GridSearchCV(nb_pipeline, parameters, cv=3) # this takes a while :/

# start time 
nb_start = time.time()

# Fit 
nb_grid.fit(X_train, y_train)

# end time 
svm_end = time.time()
print(f"Time taken to run: {round((svm_end - nb_start)/60,1)} minutes")

# Check score
nb_grid.score(X_test, y_test)



In [60]:
print('Best params: {}'.format(nb_grid.best_params_))
nb_cv_results = pd.DataFrame(nb_grid.cv_results_)
nb_cv_results.head(3)

Best params: {'clf__alpha': 1.0, 'clf__fit_prior': False, 'feats__text__tfidf__binary': 'l1', 'feats__text__tfidf__max_df': 0.5, 'feats__text__tfidf__ngram_range': (1, 1), 'feats__text__tfidf__use_idf': True}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_clf__fit_prior,param_feats__text__tfidf__binary,param_feats__text__tfidf__max_df,param_feats__text__tfidf__ngram_range,param_feats__text__tfidf__use_idf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.046393,0.002797,0.019311,0.000327,1,True,l1,0.5,"(1, 1)",False,"{'clf__alpha': 1.0, 'clf__fit_prior': True, 'f...",0.575601,0.546392,0.560137,0.560710,0.011932,58
1,0.045267,0.001148,0.020197,0.000960,1,True,l1,0.5,"(1, 1)",True,"{'clf__alpha': 1.0, 'clf__fit_prior': True, 'f...",0.597938,0.551546,0.587629,0.579038,0.019890,47
2,0.079374,0.003512,0.025655,0.000415,1,True,l1,0.5,"(1, 2)",False,"{'clf__alpha': 1.0, 'clf__fit_prior': True, 'f...",0.548110,0.530928,0.532646,0.537228,0.007727,145


In [51]:
# Support Vector Classifier

# combine features
features_count = features_union(tfidf)

# instantiate pipeline
svm_count_pipeline = Pipeline([('feats', features_count),  ('clf', LinearSVC())])

# parameter grid (3x3x2x3x7x2) combinations
parameters = {
    'feats__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'feats__text__tfidf__ngram_range': ((1, 1), (1, 2), (2, 2)), 
    'feats__text__tfidf__use_idf': (False, True),
    'clf__loss': ('hinge', 'squared_hinge'),
    'clf__C': (0.1, 0.5, 0.6, 1, 4, 5, 10, 100),
    'clf__class_weight': (None, 'balanced')                                    
}

# instantiate GridSearchCV object with pipeline and parameters with 3-folds cross-validation
svm_grid = GridSearchCV(svm_count_pipeline, parameters, cv=3)

# start time 
svm_start = time.time()

# fit
svm_grid.fit(X_train, y_train)

# end time 
svm_end = time.time()
print(f"Time taken to run: {round((svm_end - svm_start)/60,1)} minutes")

# score
svm_grid.score(X_test, y_test)

ValueError: Invalid parameter text for estimator FeatureUnion(transformer_list=[('message',
                                Pipeline(steps=[('selector',
                                                 TextSelector(key='message')),
                                                ('tfidf', TfidfVectorizer())])),
                               ('neu_scores',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='neu_scores')),
                                                ('minmax', MinMaxScaler())])),
                               ('neg_scores',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='neg_scores')),
                                                ('minmax', MinMaxScaler())])),
                               ('pos_scores',
                                Pipeline(steps=[('selector',
                                        